# Prep Dockets Detailed
2/24/24

This notebook normalizes data contained in the followup FOIA response to Admin Hearings, which provided more detail for dockets.

|Source|Description|Filename|
|---|---|---|
|Administrative Hearings|citations from January 1, 2019 to the present, related to uncleared sidewalks following snowfall, including info on respondents|[FOIA-H067483-021424-snow-clearance-fines-w-respondents.csv](../data/01-raw/FOIA-H067483-021424-snow-clearance-fines-w-respondents.csv)|

### findings
<ol>
    <li>dockets can have multiple respondents, case dispositions, hearing dates, and imposed fees per docket
    </ol>
    
I did not attempt to normalize this dataset, but instead cleaned it up and exported for future analysis

# 1. Assess Fines with Respondents Dataset

In [1]:
import pandas as pd

### review fines with respondents dataset

In [2]:
df_dockets2 = pd.read_csv("../data/01-raw/FOIA-H067483-021424-snow-dockets-detailed.csv")
df_dockets2.head()

,Respondent Full Name,Docket Number,Nov #,Violation Date,Hearing Date,Violation Address,Violation Description,Case Disposition Label,Imposed Fine Detailed
0,1010 W MADISON LLC,23DT000582,T000215159,01/31/2023,7/10/2023,1010 W MADISON,10-8-180 Snow and ice removal.,Non-Suit,$0.00
1,10-18 S HOMAN AVE HOLDINGS LLC c/o BRIAN GOTTE...,19DS41774L,241774L,01/22/2019,6/3/2019,10 S HOMAN AVE,10-8-180 Snow and ice removal.,Liable,$0.00
2,10-18 S HOMAN AVE HOLDINGS LLC c/o EV HAS LLC,19DS41774L,241774L,01/22/2019,6/3/2019,10 S HOMAN AVE,10-8-180 Snow and ice removal.,Liable,$0.00
3,"1100 E 47TH STREET LLC, C/O PETER CASSEL",22DT001042,T000207342,01/04/2022,5/23/2022,1100 E 47TH ST,10-8-180 Snow and ice removal.,Non-Suit,$0.00
4,"1101 BERWYN LLC, C/O JOHN BARRETT",23DT000559,T000220112,02/02/2023,3/20/2023,1101 W BERWYN,10-8-180 Snow and ice removal.,Non-Suit,$0.00


In [3]:
df_dockets2['Docket Number'].nunique()

2348

In [4]:
# violation descriptions
df_dockets2.groupby('Violation Description').size().reset_index()

,Violation Description,0
0,10-8-180 Snow and ice removal.,4330


In [5]:
# case dispositions
# coninuance means case is postponed or continued to a later date
# default usually means ruling in favor of city b/c defendant didn't show up at hearing or respond to violation
df_dockets2.groupby('Case Disposition Label').size().reset_index(name='n').sort_values(by='n',ascending=False)

,Case Disposition Label,n
3,Liable,1512
1,Default,1207
0,Continuance,1105
4,Non-Suit,401
5,Not Liable,64
7,Pending,30
2,Dismissed (DWP),9
6,Other,1
8,Vacated,1


In [6]:
len(df_dockets2)

4330

### identify entity relationships
messier than I thought, taking a break from this

In [7]:
# this shows that each docket can have only one address...
df_dockets2.groupby(['Docket Number','Violation Address']).ngroups

2348

# 2. Create Respondents Recordset

### clean up field names

In [11]:
df_dockets2 = df_dockets2.rename(columns={'Docket Number':'docket',
                                                          'Hearing Date':'hearing_date',
                                                          'Respondent Full Name':'respondent',
                                                          'Case Disposition Label':'case_disposition',
                                                          'Imposed Fine Detailed':'fine'})
df_dockets2 = df_dockets2[['docket','hearing_date','respondent','case_disposition','fine']].sort_values(['docket','hearing_date','respondent'])
df_dockets2.head()

,docket,hearing_date,respondent,case_disposition,fine
443,18CP093079,1/24/2019,DOE JOHN,Not Liable,$0.00
580,18DS11011L,4/24/2019,GREATER ST. TITUS MINISTRIES CHURCH OF GOD IN ...,Non-Suit,$0.00
1285,18DS16572L,1/7/2019,"CICERO HAMDAN, LLC c/o ILLINOIS SECRETARY OF S...",Liable,$50.00
1286,18DS16572L,1/7/2019,"CICERO HAMDAN, LLC c/o MOHAMMED A. NOFAL - ATT...",Liable,$50.00
1329,18DS16572L,1/7/2019,HAMDAN SUSAN,Liable,$50.00


# 3. Export for Further Analysis

In [12]:
df_dockets2.to_csv("../data/04-standardized/dockets-detailed.csv", index = False)